In [4]:
#importing libraries
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
import string

def automate_excel(file_name):
    '''This file name should have the following structure: sales_month.xlsx'''
    
    #read excel file
    excel_file=pd.read_excel(file_name)
    
    #make pivot table
    report_table = excel_file.pivot_table(index='Gender',columns='Product line',values='Total',aggfunc='sum').round(0)
    
    #splitting the month and extension from the filename
    month_and_extension = file_name.split('_')[1]
    
    #send the report table to excel file
    report_table.to_excel(f'report_{month_and_extension}',sheet_name='Report',startrow=4)
    
    #loading workbook and selecting sheet
    wb=load_workbook(f'report_{month_and_extension}')
    sheet = wb['Report']
    
    #cell reference (original spreadsheet)
    min_column = wb.active.min_column
    max_column = wb.active.max_column
    min_row = wb.active.min_row
    max_row = wb.active.max_row
    
    #adding the chart
    barchart = BarChart()

    #locate data & categories
    data = Reference(sheet, min_col=min_column+1, max_col=max_column, min_row = min_row, max_row=max_row) # including headers

    categories = Reference(sheet, min_col=min_column, max_col=max_column, min_row = min_row+1, max_row=max_row) #not including headers

    #adding data & categories 
    barchart.add_data(data, titles_from_data=True)
    barchart.set_categories(categories)

    #location chart
    sheet.add_chart(barchart, "B12")

    barchart.title = 'Sales by Product line'
    barchart.style = 2 #choose the chart style
    
    #applying formulas
    #first create alphabet list as references for cells
    alphabet = list(string.ascii_uppercase)
    excel_alphabet = alphabet[0:max_column]
    
    #sum in column B-G
    for i in excel_alphabet:
        if i!='A':
            sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
            sheet[f'{i}{max_row+1}'].style = 'Currency'

    #adding total label
    sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'
    
    #getting month name
    month_name = month_and_extension.split('.')[0]
    
    #formatting the report
    sheet['A1'] = 'Sales Report'
    sheet['A2'] =2021
    sheet['A1'].font = Font('Arial',bold=True, size=20)
    sheet['A2'].font = Font('Arial',bold=True, size=10)

    wb.save(f'report_{month_and_extension}')
    return

In [5]:
automate_excel('sales_2021.xlsx')

In [6]:
automate_excel('sales_january.xlsx')
automate_excel('sales_february.xlsx')
automate_excel('sales_march.xlsx')